In [1]:
from utils import Activities, Users, InfoParser

import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import logging
logging.basicConfig(level=logging.DEBUG)

from config_vars import month_map, month_order

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows    = None

In [3]:
data_folder = "../data/"
raws_folder = "../data/mysql_extractions/"
suffixes    = [""]#,"_GE","_TI"]

In [4]:
inii = InfoParser("../report/data_info.ini")

# Data Loading and Cleaning

### Loading Activities

In [6]:
for suffix in suffixes:
    
    activities = Activities(f"{raws_folder}activities{suffix}.csv")

    activities.drop_over_year() 
    activities.map_month(month_map,month_order) # map and sort months
    activities.save_raw() # create a copy of non cleaned dataframe
    activities.fillna() # fill some columns with 0 instead of NaN
    activities.year_to_cat()
    
    #EXPORT
    activities.df.to_csv(f"{data_folder}activities{suffix}.csv",sep='\t',index=False)
    
    print(f"activities{suffix} processed correctly.")

activities processed correctly.


### Cleaning Activities

In [7]:
activities = pd.read_csv(f"{data_folder}activities.csv",sep='\t')

# remove activities with 0 length
before = len(activities)
activities.drop(activities.query("activity_total_length == 0").index, inplace=True)
after = len(activities)

inii.info('activities_0_length',before-after)

print(f"{before-after} activities removed, {after} activities remained")

5966 activities removed, 38556 activities remained


remove activities that doesn't have a user

In [10]:
users = pd.read_csv(f"{data_folder}users.csv",sep='\t')
users['has_user'] = True
activities = activities.merge(users[['us_user','has_user']],on="us_user",how="left")

In [11]:
before = len(activities)
activities.dropna(subset=["has_user"],inplace=True)
after = len(activities)

inii.info('activities_no_user',before-after)

print(f"{before-after} activities removed, {after} activities remained")

187 activities removed, 38369 activities remained


# Export extracted data

In [12]:
# activities.df.to_csv(f"{data_folder}activities{suffix}.csv",sep='\t',index=False)
activities.to_csv(f"{data_folder}activities.csv",sep='\t',index=False)